In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

In [103]:
df = pd.read_csv('load_power.csv', usecols=['start', 'mean'])
# df['mean']
df['start'] = pd.to_datetime(df['start'], format='%Y-%m-%d %H:%M:%S')
df = df.set_index('start')
# df = df.asfreq('H')
df = df[['mean']]

def split_dataset(data):
    # Hacky: align to Mon-Fri
    train, validate, test = data[0:-24*60], data[-24*60:-24*14], data[-24*14:0]
    # train, test = data[11:-24*(7*4+1)-12], data[-24*(7*4+1)-12:-12]
    # print(train)
    # print(split(train, 24))
    # train = array(split(train, len(train)/24))
    # print(train)
    # print(test)
    # test = array(split(test, len(test)/24))
    return train, validate, test

train, validate, test = split_dataset(df)
train
# fig, ax = plt.subplots(figsize=(6, 3))
# train.plot(ax=ax, label='train')
# test.plot(ax=ax, label='test')
# ax.legend()

,mean
start,
2023-03-12 20:00:00,0.277925
2023-03-12 21:00:00,0.330584
2023-03-15 15:00:00,0.437755
2023-03-15 16:00:00,0.303670
2023-03-15 17:00:00,0.328042
...,...
2023-04-04 07:00:00,0.246629
2023-04-04 08:00:00,0.232646
2023-04-04 09:00:00,0.242139


In [ ]:
forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags = 24
             )

forecaster.fit(y=train['mean'])

In [ ]:
predictions = forecaster.predict(steps=len(test))

error_mse = mean_squared_error(
                y_true = test,
                y_pred = predictions
            )
print(f"Test error (mse): {error_mse}")
forecaster.get_feature_importance()

In [105]:
forecaster = ForecasterAutoreg(
                  regressor = RandomForestRegressor(random_state=123),
                  lags      = 15 
             )

metric, predictions_backtest = backtesting_forecaster(
                                    forecaster = forecaster,
                                    y          = df[:240]['mean'],
                                    initial_train_size = 20,
                                    fixed_train_size   = False,
                                    steps      = 3,
                                    metric     = 'mean_squared_error',
                                    refit      = True,
                                    verbose    = True
                               )

c:\Users\Antony\.virtualenvs\LoadPowerForecast-xBBx0yCS\lib\site-packages\skforecast\model_selection\model_selection.py:448: LongTrainingWarning: The forecaster will be fit 74 times. This can take substantial amounts of time. If not feasible, try with `refit = False`.
 
 You can suppress this warning using: warnings.simplefilter('ignore', category=LongTrainingWarning)
  warnings.warn(


Information of backtesting process
----------------------------------
Number of observations used for initial training: 20
Number of observations used for backtesting: 220
    Number of folds: 74
    Number of steps per fold: 3
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 1 observations.

Fold: 0
    Training:   2023-03-12 20:00:00 -- 2023-03-16 08:00:00  (n=20)
    Validation: 2023-03-16 09:00:00 -- 2023-03-16 11:00:00  (n=3)
Fold: 1
    Training:   2023-03-12 20:00:00 -- 2023-03-16 11:00:00  (n=23)
    Validation: 2023-03-16 12:00:00 -- 2023-03-16 14:00:00  (n=3)
Fold: 2
    Training:   2023-03-12 20:00:00 -- 2023-03-16 14:00:00  (n=26)
    Validation: 2023-03-16 15:00:00 -- 2023-03-16 17:00:00  (n=3)
Fold: 3
    Training:   2023-03-12 20:00:00 -- 2023-03-16 17:00:00  (n=29)
    Validation: 2023-03-16 18:00:00 -- 2023-03-16 20:00:00  (n=3)
Fold: 4
    Training:   2023-03-12 20:00:00 -- 2023-03-16 20:00:00  (n=32)
    V

  0%|          | 0/74 [00:00<?, ?it/s]c:\Users\Antony\.virtualenvs\LoadPowerForecast-xBBx0yCS\lib\site-packages\skforecast\utils\utils.py:791: UserWarning: `y` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
c:\Users\Antony\.virtualenvs\LoadPowerForecast-xBBx0yCS\lib\site-packages\skforecast\utils\utils.py:791: UserWarning: `y` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
c:\Users\Antony\.virtualenvs\LoadPowerForecast-xBBx0yCS\lib\site-packages\skforecast\utils\utils.py:854: UserWarning: `last_window` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
  1%|▏         | 1/74 [00:00<00:38,  1.90it/s]c:\Users\Antony\.virtualenvs\LoadPowerForecast-xBBx0yCS\lib\site-packages\skforecast\utils\utils.py:791: UserWarning: `y` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.

KeyError: "None of [Index([15, 16, 17, 15, 16, 17, 15, 16, 17, 15,\n       ...\n       15, 16, 17, 15, 16, 17, 15, 16, 17, 15],\n      dtype='int64', length=220)] are in the [index]"

In [72]:
url = ('https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/h2o.csv')
data = pd.read_csv(url, sep=',', header=0, names=['y', 'datetime'])

# Data preprocessing
# ==============================================================================
# data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
# data = data.set_index('datetime')
# data = data.asfreq('MS')
# data = data[['y']]
# data = data.sort_index()
data

# Train-validation dates
# ==============================================================================
end_train = '2002-01-01 23:59:00'

print(f"Train dates      : {data.index.min()} --- {data.loc[:end_train].index.max()}  (n={len(data.loc[:end_train])})")
print(f"Validation dates : {data.loc[end_train:].index.min()} --- {data.index.max()}  (n={len(data.loc[end_train:])})")

forecaster = ForecasterAutoreg(
                  regressor = RandomForestRegressor(random_state=123),
                  lags      = 15 
             )

data['y']
# metric, predictions_backtest = backtesting_forecaster(
#                                     forecaster = forecaster,
#                                     y          = data['y'],
#                                     initial_train_size = len(data.loc[:end_train]),
#                                     fixed_train_size   = False,
#                                     steps      = 10,
#                                     metric     = 'mean_squared_error',
#                                     refit      = True,
#                                     verbose    = True
#                                )

Train dates      : 0 --- 200  (n=201)
Validation dates : 201 --- 203  (n=3)


0      0.429795
1      0.400906
2      0.432159
3      0.492543
4      0.502369
         ...   
199    0.761822
200    0.649435
201    0.827887
202    0.816255
203    0.762137
Name: y, Length: 204, dtype: float64